In [1]:
import tensorflow as tf
tf.enable_eager_execution()

# FUNCTION FOR DATA AUGMENTATION
def flip(x: tf.Tensor, y:tf.Tensor) -> (tf.Tensor,tf.Tensor):
    x = tf.image.random_flip_left_right(x)
    return (x,y)

# FUNCTION FOR DATA AUGMENTATION
def crop(x: tf.Tensor, y:tf.Tensor) -> (tf.Tensor,tf.Tensor):
    x = tf.random_crop(x,size=[64,64,3])
    return (x,y)

In [2]:
MAPS = 128
noise_dim = 128

from tensorflow.keras import layers
from tensorflow.keras.initializers import RandomNormal
init = RandomNormal(mean=0.0, stddev=0.02)

def make_generator():
    seed = tf.keras.Input(shape=((noise_dim,)))
    label = tf.keras.Input(shape=((1,)))
    x = layers.Embedding(120, 120, input_length=1,name='emb')(label)
    x = layers.Flatten()(x)
    x = layers.concatenate([seed,x])
    x = layers.Dense(4*4*MAPS*8, use_bias=False)(x)
    x = layers.Reshape((4, 4, MAPS*8))(x)
    x = layers.BatchNormalization()(x)
    x = layers.ReLU()(x)
    
    x = layers.Conv2DTranspose(MAPS*4, (5, 5), strides=(2, 2), padding='same', kernel_initializer=init, use_bias=False)(x)
    x = layers.BatchNormalization()(x)
    x = layers.ReLU()(x)
    
    x = layers.Conv2DTranspose(MAPS*2, (5, 5), strides=(2, 2), padding='same', kernel_initializer=init, use_bias=False)(x)
    x = layers.BatchNormalization()(x)
    x = layers.ReLU()(x)
    
    x = layers.Conv2DTranspose(MAPS, (5, 5), strides=(2, 2), padding='same', kernel_initializer=init, use_bias=False)(x)
    x = layers.BatchNormalization()(x)
    x = layers.ReLU()(x)
    
    x = layers.Conv2DTranspose(3, (5, 5), strides=(2, 2), padding='same', kernel_initializer=init, use_bias=False, activation='tanh')(x)

    model = tf.keras.Model(inputs=[seed,label], outputs=x)    
    return model

generator = make_generator()
#generator.summary()

Instructions for updating:
Colocations handled automatically by placer.


In [3]:
GNOISE = 0.25

def make_discriminator():
    image = tf.keras.Input(shape=((64,64,3)))
    label = tf.keras.Input(shape=((1,)))
    x = layers.Embedding(120, 64*64, input_length=1)(label)
    x = layers.Reshape((64,64,1))(x)
    x = layers.concatenate([image,x])
    
    x = layers.Conv2D(MAPS, (5, 5), strides=(2, 2), padding='same', kernel_initializer=init, use_bias=False)(x)
    x = layers.BatchNormalization()(x)
    #x = layers.GaussianNoise(GNOISE)(x)
    x = layers.LeakyReLU()(x)

    x = layers.Conv2D(MAPS*2, (5, 5), strides=(2, 2), padding='same', kernel_initializer=init, use_bias=False)(x)
    x = layers.BatchNormalization()(x)
    #x = layers.GaussianNoise(GNOISE)(x)
    x = layers.LeakyReLU()(x)

    x = layers.Conv2D(MAPS*4, (5, 5), strides=(2, 2), padding='same', kernel_initializer=init, use_bias=False)(x)
    x = layers.BatchNormalization()(x)
    #x = layers.GaussianNoise(GNOISE)(x)
    x = layers.LeakyReLU()(x)

    x = layers.Conv2D(MAPS*8, (5, 5), strides=(2, 2), padding='same', kernel_initializer=init, use_bias=False)(x)
    x = layers.BatchNormalization()(x)
    #x = layers.GaussianNoise(GNOISE)(x)
    x = layers.LeakyReLU()(x)
    
    x = layers.Flatten()(x)
    x = layers.Dense(121, activation='sigmoid')(x)
    x2 = layers.Dense(1, activation='linear')(x)
    
    model = tf.keras.Model(inputs=[image,label], outputs=[x,x2])
    return model

discriminator = make_discriminator()

In [4]:
from tensorflow.contrib.eager.python import tfe

# RaLS Discriminator Loss
def RaLS_errD(fake,real):
    return (tf.reduce_mean( (real - tf.reduce_mean(fake,0) - tf.ones_like(real))**2,0 )
        + tf.reduce_mean( (fake - tf.reduce_mean(real,0) + tf.ones_like(real))**2,0 ) )/2.

# RaLS Generator Loss
def RaLS_errG(fake,real):
    return (tf.reduce_mean( (real - tf.reduce_mean(fake,0) + tf.ones_like(real))**2,0 )
        + tf.reduce_mean( (fake - tf.reduce_mean(real,0) - tf.ones_like(real))**2,0 ) )/2.

# OPTIMIZER - ADAM
learning_rate = tfe.Variable(0.0002)
generator_optimizer = tf.train.AdamOptimizer(learning_rate=learning_rate, beta1=0.5)
discriminator_optimizer = tf.train.AdamOptimizer(learning_rate=learning_rate, beta1=0.5)

In [6]:
EPOCHS = 250
num_examples = 16

#@ tf.function
def train_step(images,generator,discriminator,generator_optimizer,discriminator_optimizer):
        
    bce = tf.keras.losses.BinaryCrossentropy(from_logits=True,label_smoothing=0.4)
    bce2 = tf.keras.losses.BinaryCrossentropy(from_logits=False,label_smoothing=0.4)
    noise = tf.random.normal((32,128)) # update noise_dim here
    labs = tf.cast(120*tf.random.uniform((32,)),tf.int32)
    
    # USE GRADIENT TAPE TO CALCULATE GRADIENTS
    with tf.GradientTape() as gen_tape, tf.GradientTape() as disc_tape:       
        generated_images = generator([noise,labs], training=True)
        real_cat, real_output = discriminator([images[0],images[1]], training=True)
        fake_cat, fake_output = discriminator([generated_images,labs], training=True)
    
        # GENERATOR LOSS 
        gen_loss = (tf.reduce_mean( (real_output - tf.reduce_mean(fake_output,0) + tf.ones_like(real_output))**2,0 )
        + tf.reduce_mean( (fake_output - tf.reduce_mean(real_output,0) - tf.ones_like(real_output))**2,0 ) )/2.
        
        # DISCRIMINATOR LOSS
        disc_loss = bce(tf.ones_like(real_output), real_output) + bce(tf.zeros_like(fake_output), fake_output)           
        real_cat2 = tf.one_hot(tf.cast(images[1],tf.int32),121,dtype=tf.int32)
        fake_cat2 = tf.one_hot(120*tf.ones((32,),tf.int32),121,dtype=tf.int32)
        disc_loss += bce2(real_cat2,real_cat) + bce2(fake_cat2,fake_cat) 
        
    # BACK PROPAGATE ERROR
    gradients_of_generator = gen_tape.gradient(gen_loss, generator.trainable_variables)
    gradients_of_discriminator = disc_tape.gradient(disc_loss, discriminator.trainable_variables)
    generator_optimizer.apply_gradients(zip(gradients_of_generator, generator.trainable_variables))
    discriminator_optimizer.apply_gradients(zip(gradients_of_discriminator, discriminator.trainable_variables))
       
    return gen_loss, disc_loss

print('Training started. Displaying every '+str(DISPLAY_EVERY)+'th epoch.')
train(ds, EPOCHS)

NameError: name 'DISPLAY_EVERY' is not defined